In [8]:
import requests

url = "https://ckan.pbh.gov.br/dataset/ec3efaac-0ca6-4846-9e32-0ffff2d76dbb/resource/4189b28e-7592-474b-8160-e39731315c27/download/20250401_atividade_economica.csv"
output_path = "../data/20250401_atividade_economica.csv"

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    with open(output_path, 'wb') as f:
        f.write(response.content)
    print(f"Arquivo salvo como: {output_path}")
else:
    print(f"Erro ao baixar: {response.status_code}")

Arquivo salvo como: ../data/20250401_atividade_economica.csv


### Limpar Dados

In [9]:
import pandas as pd

df = pd.read_csv("../data/20250401_atividade_economica.csv", sep=";", encoding="utf-8")
print(df.columns)
print(f"Registros iniciais: {len(df)}")

Index(['ID_ATIV_ECON_ESTABELECIMENTO', 'CNAE_PRINCIPAL',
       'DESCRICAO_CNAE_PRINCIPAL', 'CNAE', 'DATA_INICIO_ATIVIDADE',
       'NATUREZA_JURIDICA', 'PORTE_EMPRESA', 'AREA_UTILIZADA', 'IND_SIMPLES',
       'IND_MEI', 'IND_POSSUI_ALVARA', 'TIPO_UNIDADE', 'FORMA_ATUACAO',
       'DESC_LOGRADOURO', 'NOME_LOGRADOURO', 'NUMERO_IMOVEL', 'COMPLEMENTO',
       'NOME_BAIRRO', 'NOME', 'NOME_FANTASIA', 'CNPJ', 'GEOMETRIA'],
      dtype='object')
Registros iniciais: 534880


In [10]:
df['DESCRICAO_CNAE_PRINCIPAL'] = df['DESCRICAO_CNAE_PRINCIPAL'].fillna("").str.upper()

keywords = ["BAR", "RESTAURANTE", "BARES", "RESTAURANTES"]

filter = df['DESCRICAO_CNAE_PRINCIPAL'].apply(lambda x: any(k in x for k in keywords))
df_final = df[filter].copy()

df_final['ENDERECO'] = (
    df_final['DESC_LOGRADOURO'].fillna('') + ' ' +
    df_final['NOME_LOGRADOURO'].fillna('') + ', ' +
    df_final['NUMERO_IMOVEL'].fillna('S/N').astype(str) + ' - ' +
    df_final['NOME_BAIRRO'].fillna('') +
    ' , Belo Horizonte, MG, Brasil'
)
df_final['ENDERECO'].dropna().unique()

print(f"Registros restantes: {len(df_final)}")

Registros restantes: 13801


In [11]:
df_final["NOME"] = df_final["NOME_FANTASIA"].combine_first(df_final["NOME"])
df_final = df_final[['NOME', 'DATA_INICIO_ATIVIDADE', 'IND_POSSUI_ALVARA', 'ENDERECO']]
print(df_final.head())

                                       NOME DATA_INICIO_ATIVIDADE  \
17                 APARECIDA MARIA DE SOUZA            01-07-1993   
19          PIZZARIA E CHURRASCARIA VARANDA            15-10-1993   
29                            RABBIT BURGER            02-05-1994   
172             TATU REI DO ANGU A  BAHIANA            24-11-1993   
196  ROD BITS COMERCIO E REPRESENTACAO LTDA            01-06-1994   

    IND_POSSUI_ALVARA                                           ENDERECO  
17                NÃO  RUA DESEMBARGADOR REIS ALVES, 90 - BAIRRO DAS ...  
19                NÃO  RUA LUIZ PONGELUPE, 290 - CARDOSO , Belo Horiz...  
29                NÃO  AVE RESSACA, 118 - PADRE EUSTAQUIO , Belo Hori...  
172               SIM  RUA DESEMBARGADOR RIBEIRO DA LUZ, 135 - BARREI...  
196               NÃO  AVE ELIAS ANTONIO ISSA, 288 - LETICIA , Belo H...  


### Geocodificação

In [4]:
%pip install geopy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="pbh-geocodificador", timeout=5)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=3, error_wait_seconds=5, swallow_exceptions=False)

In [5]:
def convert_coordinates(address: str):
    try:
        full_address = f"{address}, Belo Horizonte, MG, Brasil"
        location = geocode(full_address)
        if location:
            return pd.Series([location.latitude, location.longitude])
    except Exception as e:
        print(f"[ERRO] '{address}' → {e.__class__.__name__}: {e}")
    return pd.Series([None, None])

In [12]:
df_final[['LATITUDE', 'LONGITUDE']] = df_final['ENDERECO'].apply(convert_coordinates)
print(df_final.head())

RateLimiter caught an error, retrying (0/3 tries). Called with (*('AVE AUGUSTO DE LIMA, 315 - CENTRO , Belo Horizonte, MG, Brasil, Belo Horizonte, MG, Brasil',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 446, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 441, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
 

[ERRO] 'AVE AUGUSTO DE LIMA, 315 - CENTRO , Belo Horizonte, MG, Brasil' → GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=AVE+AUGUSTO+DE+LIMA%2C+315+-+CENTRO+%2C+Belo+Horizonte%2C+MG%2C+Brasil%2C+Belo+Horizonte%2C+MG%2C+Brasil&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=5)"))


RateLimiter caught an error, retrying (0/3 tries). Called with (*('AVE NOSSA SENHORA DA PIEDADE, 179 - SAO GONCALO , Belo Horizonte, MG, Brasil, Belo Horizonte, MG, Brasil',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 446, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 441, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, 

[ERRO] 'RUA FERNANDES TOURINHO, 292 - SAVASSI , Belo Horizonte, MG, Brasil' → GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=RUA+FERNANDES+TOURINHO%2C+292+-+SAVASSI+%2C+Belo+Horizonte%2C+MG%2C+Brasil%2C+Belo+Horizonte%2C+MG%2C+Brasil&format=json&limit=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ea3bc50a530>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))


RateLimiter caught an error, retrying (0/3 tries). Called with (*('RUA MARQUES DE BARBACENA, 126 - SAUDADE , Belo Horizonte, MG, Brasil, Belo Horizonte, MG, Brasil',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/usr/lib/python3/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python3.10/socket.py", line 955, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 700, in urlopen
    httplib_response = self._make_request(
  File "/usr/lib/python3/dist-packages/urllib3/c

                                       NOME DATA_INICIO_ATIVIDADE  \
17                 APARECIDA MARIA DE SOUZA            01-07-1993   
19          PIZZARIA E CHURRASCARIA VARANDA            15-10-1993   
29                            RABBIT BURGER            02-05-1994   
172             TATU REI DO ANGU A  BAHIANA            24-11-1993   
196  ROD BITS COMERCIO E REPRESENTACAO LTDA            01-06-1994   

    IND_POSSUI_ALVARA                                           ENDERECO  \
17                NÃO  RUA DESEMBARGADOR REIS ALVES, 90 - BAIRRO DAS ...   
19                NÃO  RUA LUIZ PONGELUPE, 290 - CARDOSO , Belo Horiz...   
29                NÃO  AVE RESSACA, 118 - PADRE EUSTAQUIO , Belo Hori...   
172               SIM  RUA DESEMBARGADOR RIBEIRO DA LUZ, 135 - BARREI...   
196               NÃO  AVE ELIAS ANTONIO ISSA, 288 - LETICIA , Belo H...   

      LATITUDE  LONGITUDE  
17  -19.962039 -44.000033  
19         NaN        NaN  
29         NaN        NaN  
172 -19.973201 -

In [ ]:
df_final = df_final.dropna(subset=["LATITUDE", "LONGITUDE"]).reset_index(drop=True)
df_final = df_final.reset_index(drop=True)
df_final["id"] = df_final.index
df_final.to_csv("../data/bares_restaurantes_geocodificados.csv", index=False, encoding="utf-8")

### Task Extra: Comida di Buteco

In [11]:
%pip install Unidecode

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [41]:
import pandas as pd

df_extra = pd.read_csv("../data/comida_di_buteco_2025.csv", sep=",", encoding="utf-8")
print(df_extra.columns)
print(f"Registros iniciais: {len(df_extra)}")

df_general = pd.read_csv("../data/bares_restaurantes_geocodificados.csv", sep=",", encoding="utf-8")
print(df_general.columns)
print(f"Registros iniciais: {len(df_general)}")

Index(['NOME', 'PRATO', 'DESCRICAO', 'ENDERECO', 'TELEFONE', 'IMG_URL'], dtype='object')
Registros iniciais: 124
Index(['NOME', 'DATA_INICIO_ATIVIDADE', 'IND_POSSUI_ALVARA', 'ENDERECO',
       'LATITUDE', 'LONGITUDE'],
      dtype='object')
Registros iniciais: 9771


In [ ]:
from unidecode import unidecode
import unicodedata
import pandas as pd
import re

# Função de normalização
def normalize_address(endereco):
    endereco = unicodedata.normalize('NFD', endereco)
    endereco = ''.join(c for c in endereco if unicodedata.category(c) != 'Mn').lower()
    endereco = endereco.replace("belo horizonte", "").replace("mg", "").replace("brasil", "")
    endereco = re.sub(r"[\|–\-\,]", " ", endereco)
    endereco = re.sub(r"\s+", " ", endereco).strip()
    return endereco

# Normaliza nomes e endereços
df_extra["nome_normalizado"] = df_extra["NOME"].apply(lambda x: unidecode(x).lower())
df_extra["endereco_normalizado"] = df_extra["ENDERECO"].apply(normalize_address)

df_general["nome_normalizado"] = df_general["NOME"].apply(lambda x: unidecode(x).lower())
df_general["endereco_normalizado"] = df_general["ENDERECO"].apply(normalize_address)

# Subconjuntos com info a ser herdada do df_general
inherited_columns = ["ENDERECO", "LATITUDE", "LONGITUDE", "id"]
df_general_names = df_general[["nome_normalizado"] + inherited_columns]
df_general_addresses = df_general[["endereco_normalizado"] + inherited_columns]

# Merge por nome
by_name = df_extra.merge(df_general_names, on="nome_normalizado", how="inner", suffixes=('', '_geral'))

# Merge por endereço
by_address = df_extra.merge(df_general_addresses, on="endereco_normalizado", how="inner", suffixes=('', '_geral'))

# Juntar e tirar duplicatas
df_merged = pd.concat([by_name, by_address])
df_merged = df_merged.drop_duplicates(subset=["nome_normalizado", "endereco_normalizado"])

# Substitui apenas o ENDERECO por ENDERECO_geral e adiciona LATITUDE/LONGITUDE
df_merged["ENDERECO"] = df_merged["ENDERECO_geral"]

# Seleciona colunas finais: tudo de df_extra original + LATITUDE/LONGITUDE
result = [col for col in df_extra.columns if col not in ["endereco_normalizado", "nome_normalizado"]]
final_result = df_merged[result + ["LATITUDE", "LONGITUDE", "id"]]


In [ ]:
final_result.to_csv("../data/comida_di_buteco_corrigido.csv", index=False, encoding="utf-8")
